In [44]:
import math
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from pprint import pprint

class DecisionTreeClassifier:
    def __init__(self):
        self.tree = None

    # Hàm tính toán entropy của nhãn
    def entropy(self, y):
        elements, counts = np.unique(y, return_counts=True)
        entropy = -sum((counts[i]/sum(counts)) * math.log2(counts[i]/sum(counts)) for i in range(len(elements)))
        return entropy

    # Hàm tính toán thông tin thu được của thuộc tính
    def information_gain(self, X, y, attribute):
        total_entropy = self.entropy(y)
        values, counts = np.unique(X[attribute], return_counts=True)
        weighted_entropy = sum((counts[i]/sum(counts)) * self.entropy(y[X[attribute] == values[i]]) for i in range(len(values)))
        return total_entropy - weighted_entropy

    # Hàm tìm thuộc tính tốt nhất để chia
    def best_split(self, X, y):
        information_gains = [self.information_gain(X, y, attribute) for attribute in X.columns]
        best_attribute = X.columns[np.argmax(information_gains)]
        return best_attribute

    # Hàm huấn luyện mô hình
    def fit(self, X, y):
        # Nếu tất cả nhãn đều giống nhau thì trả về nhãn đó
        if len(np.unique(y)) == 1:
            return y.iloc[0]
        # Nếu không còn thuộc tính để chia thì trả về nhãn có tần suất lớn nhất
        elif len(X.columns) == 0:
            return y.mode()[0]
        else:
            # Tìm thuộc tính tốt nhất để chia
            best_attribute = self.best_split(X, y)
            tree = {best_attribute: {}}
            # Lặp qua các giá trị của thuộc tính tốt nhất và tạo cây con
            for value in np.unique(X[best_attribute]):
                sub_X = X[X[best_attribute] == value].drop(columns=[best_attribute])
                sub_y = y[X[best_attribute] == value]
                subtree = self.fit(sub_X, sub_y)
                tree[best_attribute][value] = subtree
            self.tree = tree
            return tree

    # Hàm dự đoán nhãn cho một mẫu
    def predict_one(self, x, tree):
        if not isinstance(tree, dict):
            return tree
        attribute = list(tree.keys())[0]
        if x[attribute] in tree[attribute]:
            return self.predict_one(x, tree[attribute][x[attribute]])
        else:
            return random.choice(list(tree[attribute].values()))

    # Hàm dự đoán nhãn cho tập dữ liệu
    def predict(self, X):
        return X.apply(lambda x: self.predict_one(x, self.tree), axis=1)

    # Hàm tính toán độ chính xác của mô hình
    def accuracy(self, y_true, y_pred):
        return sum(y_true == y_pred) / len(y_true)



In [45]:
file_path = '/kaggle/input/team2-dataset/team2_dataset/chap10.xlsx'
df = pd.read_excel(file_path)

# Chia dữ liệu thành đặc trưng (X) và nhãn (y)
X = df.drop(columns=['Play'])
y = df['Play']

# Khởi tạo và huấn luyện cây quyết định
tree = DecisionTreeClassifier()
tree.fit(X, y)

# Dự đoán nhãn
y_pred = tree.predict(X)

# Đánh giá mô hình cây quyết định
accuracy = tree.accuracy(y, y_pred)

# Dùng pprint in cây quyết định dạng nhánh
print("Decision Tree:")
pprint(tree.tree, width=1)

# Accuracy
print("\nAccuracy: ", accuracy)
# Accuracy convert nhaaaaaaa !!!!!
print("\n ***Kết luận => Accuracy: {:.2f}%".format(accuracy * 100))

Decision Tree:
{'Outlook': {'overcast': 'yes',
             'rainy': {'Windy': {False: 'yes',
                                 True: 'no'}},
             'sunny': {'Humidity': {'high': 'no',
                                    'normal': 'yes'}}}}

Accuracy:  1.0

 ***Kết luận => Accuracy: 100.00%
